In [3]:
from cmath import cos, sin
from math import radians
import cv2
import os
import random
import numpy as np

categories = [
    {"id": 0, "name": "2_of_clubs"},
    {"id": 1, "name": "2_of_diamonds"},
    {"id": 2, "name": "2_of_hearts"},
    {"id": 3, "name": "2_of_spades"},
    {"id": 4, "name": "3_of_clubs"},
    {"id": 5, "name": "3_of_diamonds"},
    {"id": 6, "name": "3_of_hearts"},
    {"id": 7, "name": "3_of_spades"},
    {"id": 8, "name": "4_of_clubs"},
    {"id": 9, "name": "4_of_diamonds"},
    {"id": 10, "name": "4_of_hearts"},
    {"id": 11, "name": "4_of_spades"},
    {"id": 12, "name": "5_of_clubs"},
    {"id": 13, "name": "5_of_diamonds"},
    {"id": 14, "name": "5_of_hearts"},
    {"id": 15, "name": "5_of_spades"},
    {"id": 16, "name": "6_of_clubs"},
    {"id": 17, "name": "6_of_diamonds"},
    {"id": 18, "name": "6_of_hearts"},
    {"id": 19, "name": "6_of_spades"},
    {"id": 20, "name": "7_of_clubs"},
    {"id": 21, "name": "7_of_diamonds"},
    {"id": 22, "name": "7_of_hearts"},
    {"id": 23, "name": "7_of_spades"},
    {"id": 24, "name": "8_of_clubs"},
    {"id": 25, "name": "8_of_diamonds"},
    {"id": 26, "name": "8_of_hearts"},
    {"id": 27, "name": "8_of_spades"},
    {"id": 28, "name": "9_of_clubs"},
    {"id": 29, "name": "9_of_diamonds"},
    {"id": 30, "name": "9_of_hearts"},
    {"id": 31, "name": "9_of_spades"},
    {"id": 32, "name": "10_of_clubs"},
    {"id": 33, "name": "10_of_diamonds"},
    {"id": 34, "name": "10_of_hearts"},
    {"id": 35, "name": "10_of_spades"},
    {"id": 36, "name": "ace_of_clubs"},
    {"id": 37, "name": "ace_of_diamonds"},
    {"id": 38, "name": "ace_of_hearts"},
    {"id": 39, "name": "ace_of_spades"},
    {"id": 40, "name": "ace_of_spades2"},
    {"id": 41, "name": "black_joker"},
    {"id": 42, "name": "jack_of_clubs"},
    {"id": 43, "name": "jack_of_clubs2"},
    {"id": 44, "name": "jack_of_diamonds"},
    {"id": 45, "name": "jack_of_diamonds2"},
    {"id": 46, "name": "jack_of_hearts"},
    {"id": 47, "name": "jack_of_hearts2"},
    {"id": 48, "name": "jack_of_spades"},
    {"id": 49, "name": "jack_of_spades2"},
    {"id": 50, "name": "king_of_clubs"},
    {"id": 51, "name": "king_of_clubs2"},
    {"id": 52, "name": "king_of_diamonds"},
    {"id": 53, "name": "king_of_diamonds2"},
    {"id": 54, "name": "king_of_hearts"},
    {"id": 55, "name": "king_of_hearts2"},
    {"id": 56, "name": "king_of_spades"},
    {"id": 57, "name": "king_of_spades2"},
    {"id": 58, "name": "queen_of_clubs"},
    {"id": 59, "name": "queen_of_clubs2"},
    {"id": 60, "name": "queen_of_diamonds"},
    {"id": 61, "name": "queen_of_diamonds2"},
    {"id": 62, "name": "queen_of_hearts"},
    {"id": 63, "name": "queen_of_hearts2"},
    {"id": 64, "name": "queen_of_spades"},
    {"id": 65, "name": "queen_of_spades2"},
]

label_to_id = {category["name"]: category["id"] for category in categories}

def generate_random_polygon(image, min_area):
    height, width = image.shape[:2]
    points = np.random.randint(0, [width, height], (3, 2))
    while True:
        area = 0.5 * np.abs(np.cross(points[1] - points[0], points[2] - points[0]))
        if area >= min_area:
            return points.reshape((-1, 1, 2))
        points = np.random.randint(0, [width, height], (3, 2))

def adjust_contrast_brightness(image):
    alpha = random.uniform(0.5, 1.5)
    beta = random.randint(-50, 50)
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted_image

def add_triangle(image, num_shadows, min_area, transparency):
    image_with_shadow = np.copy(image)
    
    for _ in range(num_shadows):
        temp_img = np.copy(image)
        polygon = generate_random_polygon(image, min_area)
        shadow_color = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))  
        
        cv2.fillPoly(temp_img, [polygon], shadow_color)
        cv2.addWeighted(image_with_shadow, transparency, temp_img, transparency, 0, image_with_shadow)
    
    return image_with_shadow

def shear_image(image, shear_factor=0.0):
    rows, cols, _ = image.shape
    M = np.array([[1, shear_factor, 0], [0, 1, 0]])
    sheared_img = cv2.warpAffine(image, M, (cols, rows))
    
    if shear_factor > 0:
        sheared_img = sheared_img[:, int(cols*shear_factor):]
    elif shear_factor < 0:
        sheared_img = sheared_img[:, :int(cols*(1+shear_factor))]
    
    return sheared_img

def process_image(image, min_area, num_shadows, shear_factor, poly_transparency):
    image = add_triangle(image, num_shadows=num_shadows, min_area=min_area, transparency=poly_transparency)
    image = adjust_contrast_brightness(image)
    image = shear_image(image, shear_factor=shear_factor)
    
    return image

def gen_background():
    backgrounds_dir = "bg_base_imgs"
    backgrounds = os.listdir(backgrounds_dir)
    selected_background_filename = random.choice(backgrounds)
    print(selected_background_filename)
    selected_background_path = os.path.join(backgrounds_dir, selected_background_filename)
    background_img = cv2.imread(selected_background_path)
    processed_image = process_image(background_img, 5000, 7, 0.2, 0.5)

    return processed_image


def fix_canvas(image, img_size):
    canvas_width, canvas_height = img_size
    canvas_size = max(canvas_width, canvas_height, image.shape[1], image.shape[0])
    blank_canvas = np.zeros((canvas_size, canvas_size, 4), dtype=np.uint8)

    x_offset = (canvas_size - image.shape[1]) // 2
    y_offset = (canvas_size - image.shape[0]) // 2

    x_offset = max(0, x_offset)
    y_offset = max(0, y_offset)

    blank_canvas[y_offset:y_offset+image.shape[0], x_offset:x_offset+image.shape[1], :] = image

    return blank_canvas

def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))

    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]

    if len(image.shape) == 2:
        rotated_image = cv2.warpAffine(image, M, (new_w, new_h))
    elif image.shape[2] == 4:
        alpha_channel = image[:, :, 3]
        rotated_alpha = cv2.warpAffine(alpha_channel, M, (new_w, new_h))
        rotated_image = cv2.warpAffine(image[:, :, :3], M, (new_w, new_h))
        rotated_image = cv2.merge([rotated_image, rotated_alpha])
    else:
        rotated_image = cv2.warpAffine(image, M, (new_w, new_h))

    return rotated_image

def random_zoom(image, zoom_factor):
    new_w = int(image.shape[1] * zoom_factor)
    new_h = int(image.shape[0] * zoom_factor)
    resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    return resized_image

def random_shear(image, shear_factor_x, shear_factor_y):
    (h, w) = image.shape[:2]
    M = np.array([[1, shear_factor_x, 0],
                  [shear_factor_y, 1, 0]], dtype=float)

    sheared_image = cv2.warpAffine(image, M, (w + int(shear_factor_x * h), h + int(shear_factor_y * w)))

    return sheared_image

def random_shadows(image):
    alpha_channel = image[:, :, 3]
    image_rgb = image[:, :, :3]
    shadow_mask = np.zeros(image_rgb.shape[:2], dtype=np.uint8)

    num_shadows = random.randint(3, 9)
    for _ in range(num_shadows):
        x = random.randint(0, image_rgb.shape[1] // 4)
        y = random.randint(0, image_rgb.shape[0] // 4)
        w = random.randint(image_rgb.shape[1] // 4, image_rgb.shape[1] // 2)
        h = random.randint(image_rgb.shape[0] // 4, image_rgb.shape[0] // 2)

        w = min(w, image_rgb.shape[1] - x)
        h = min(h, image_rgb.shape[0] - y)

        intensity = random.randint(30, 180)
        gradient = np.linspace(intensity, 0, max(w, h)).astype(np.uint8)
        shadow = np.zeros((h, w), dtype=np.uint8)
        for i in range(h):
            shadow[i, :] = gradient[:w]

        shadow_mask[y:y+h, x:x+w] = shadow

    shadow_mask_bgr = cv2.cvtColor(shadow_mask, cv2.COLOR_GRAY2BGR)
    shadowed_image_rgb = cv2.addWeighted(image_rgb, 1, shadow_mask_bgr, -0.5, 0)
    shadowed_image = cv2.merge((shadowed_image_rgb, alpha_channel))

    return shadowed_image

def random_noise(image):
    height, width, _ = image.shape
    noise = np.random.normal(0.05, .35, (height, width, 3)).astype(np.uint8)

    noisy_image = image.copy()
    noisy_image[:, :, :3] = np.clip(noisy_image[:, :, :3] + noise, 0, 255).astype(np.uint8)

    return noisy_image

def get_bbox(image):
    alpha_channel = image[:, :, 3]
    _, binary_alpha = cv2.threshold(alpha_channel, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary_alpha, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return 0, 0, image.shape[1], image.shape[0]

    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)

    return x, y, w, h

def perf_augmentation(image_path, angle, zoom, shear_x, shear_y, img_size):
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    image_on_canvas = fix_canvas(image, img_size)

    rotated_image = rotate_image(image_on_canvas, angle)
    zoomed_image = random_zoom(rotated_image, zoom)
    sheared_image = random_shear(zoomed_image, shear_x, shear_y)
    x0, y0, w, h = get_bbox(sheared_image)

    norm_x_center = (x0 + w / 2) / img_size[1]
    norm_y_center = (y0 + h / 2) / img_size[0]
    norm_width = w / img_size[1]
    norm_height = h / img_size[0]

    cropped_image = sheared_image[y0:y0+h, x0:x0+w]
    
    shadowed_image = random_shadows(cropped_image)
    final_image = random_noise(shadowed_image)

    return final_image, (norm_x_center, norm_y_center, norm_width, norm_height)

def gen_aug_params():
    random_angle = random.uniform(-45, 45)
    random_zoom = random.uniform(0.15, 0.25)
    random_shear_x = random.uniform(-0.25, 0.25)
    random_shear_y = random.uniform(-0.25, 0.25)
    return random_angle, random_zoom, random_shear_x, random_shear_y

def normalize_bbox(bbox, img_size):
    x, y, w, h = bbox
    img_width, img_height = img_size
    norm_x_center = (x + w / 2) / img_width
    norm_y_center = (y + h / 2) / img_height
    norm_width = w / img_width
    norm_height = h / img_height
    return norm_x_center, norm_y_center, norm_width, norm_height

def gen_card_images(num_cards, img_size):
    card_images = []
    card_masks = []
    card_labels = []
    card_polygons = []
    aug_params = gen_aug_params()

    for _ in range(num_cards):
        card_path = random.choice(os.listdir('cards'))
        card_label = os.path.splitext(card_path)[0]
        card_image_path = f'cards/{card_path}'

        augmented_card, bbox = perf_augmentation(card_image_path, *aug_params, img_size)
        card_mask = np.zeros(augmented_card.shape[:2], dtype=np.uint8)
        card_mask[augmented_card[:, :, 3] > 0] = 255
        polygon_points = map_poly(card_mask)

        card_images.append(augmented_card)
        card_masks.append(bbox)
        card_labels.append(label_to_id.get(card_label, -1))
        card_polygons.append(polygon_points)

    return card_images, card_masks, card_labels, card_polygons

def card_overlay(card_images, card_bboxes, card_polygons, background, card_labels):
    annotations = []
    background_height, background_width = background.shape[:2]

    if background.shape[2] == 3:
        background = cv2.cvtColor(background, cv2.COLOR_BGR2BGRA)

    start_x = random.randint(30, 150)
    start_y = random.randint(30, 150)
    rotation_angle = random.randint(-10, 10)

    for idx, (card_image, bbox, polygon, label) in enumerate(zip(card_images, card_bboxes, card_polygons, card_labels)):
        overlap_x = random.randint(25, 30)
        overlap_y = random.randint(15, 20)
        
        x_offset = start_x + idx * overlap_x
        y_offset = start_y + idx * overlap_y

        card_center = (card_image.shape[1] // 2, card_image.shape[0] // 2)
        rotated_card_image = rotate_image(card_image, rotation_angle * idx)
        
        end_x = min(x_offset + rotated_card_image.shape[1], background_width)
        end_y = min(y_offset + rotated_card_image.shape[0], background_height)

        alpha_s = rotated_card_image[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s

        for c in range(0, 3):
            background[y_offset:end_y, x_offset:end_x, c] = (alpha_s * rotated_card_image[:, :, c] +
                                                            alpha_l * background[y_offset:end_y, x_offset:end_x, c])
        transformed_polygon = transform_poly(polygon, x_offset, y_offset, rotation_angle * idx, card_center)

        annotations.append({
            "category_id": label,
            "bbox": bbox,
            "segmentation": transformed_polygon
        })

    return background, annotations

def normalize_poly(polygon, image_width, image_height):
    return [(x / image_width, y / image_height) for x, y in polygon]

def map_poly(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return []

    largest_contour = max(contours, key=cv2.contourArea)

    epsilon = 0.01 * cv2.arcLength(largest_contour, True)
    approx = cv2.approxPolyDP(largest_contour, epsilon, True)
    return [(int(point[0][0]), int(point[0][1])) for point in approx]

def transform_poly(polygon, x_offset, y_offset, rotation_angle, card_center):
    transformed_polygon = []
    for x, y in polygon:
        x_rotated, y_rotated = rotate_point(x, y, rotation_angle, card_center[0], card_center[1])

        x_translated = x_rotated + x_offset
        y_translated = y_rotated + y_offset

        transformed_polygon.append((x_translated, y_translated))

    return transformed_polygon


def rotate_point(x, y, angle, cx, cy):
    angle = radians(angle)
    x_rotated = cos(angle) * (x - cx) - sin(angle) * (y - cy) + cx
    y_rotated = sin(angle) * (x - cx) + cos(angle) * (y - cy) + cy
    return x_rotated, y_rotated


def seg_annotations(base_name, annotations, labels_dir):
    os.makedirs(labels_dir, exist_ok=True)
    file_path = os.path.join(labels_dir, f'{base_name}.txt')

    with open(file_path, 'w') as file:
        for annotation in annotations:
            class_index = annotation['category_id']
            points = annotation['segmentation']
            line = f"{class_index} " + " ".join(f"{round(x.real, 6)} {round(y.real, 6)}" for x, y in points)
            file.write(line + "\n")

    print(f"Saved YOLO annotations to {file_path}")



def gen_dataset(num_images=10000, img_size=(512, 512)):
    dir = 'D:/yolo/data/images/val/'
    
    for i in range(num_images):
        base_name = f'image_{i:06d}'
        background = gen_background()
        resized_background = cv2.resize(background, img_size)
        #num_cards = random.randint(1, 5)
        num_cards = 1
        card_images, card_masks, card_labels, card_polygons = gen_card_images(num_cards, img_size)

        final_image, transformed_annotations = card_overlay(card_images, card_masks, card_polygons, resized_background, card_labels)

        for annotation in transformed_annotations:
            transformed_polygon = annotation['segmentation']
            normalized_polygon = normalize_poly(transformed_polygon, img_size[0], img_size[1])
            annotation['segmentation'] = normalized_polygon

        os.makedirs(dir, exist_ok=True)
        image_path = os.path.join(dir, f'{base_name}.png')
        cv2.imwrite(image_path, final_image)
        print(f"Saved {image_path}")

        seg_annotations(base_name, transformed_annotations, labels_dir='D:/yolo/data/labels/val/')

In [4]:
gen_dataset()

pexels-karolina-grabowska-4226882.jpg
Saved D:/yolo/data/images/val/image_000000.png
Saved YOLO annotations to D:/yolo/data/labels/val/image_000000.txt
charlie-lVH3nplcE50-unsplash.jpg
Saved D:/yolo/data/images/val/image_000001.png
Saved YOLO annotations to D:/yolo/data/labels/val/image_000001.txt
pexels-dhally-romy-12656497.jpg
Saved D:/yolo/data/images/val/image_000002.png
Saved YOLO annotations to D:/yolo/data/labels/val/image_000002.txt
joakim-nadell-cFs_R1c71CI-unsplash.jpg
Saved D:/yolo/data/images/val/image_000003.png
Saved YOLO annotations to D:/yolo/data/labels/val/image_000003.txt
pexels-dids-3794359.jpg
Saved D:/yolo/data/images/val/image_000004.png
Saved YOLO annotations to D:/yolo/data/labels/val/image_000004.txt
christin-hume-hBuwVLcYTnA-unsplash.jpg
Saved D:/yolo/data/images/val/image_000005.png
Saved YOLO annotations to D:/yolo/data/labels/val/image_000005.txt
pexels-roman-odintsov-4925691.jpg
Saved D:/yolo/data/images/val/image_000006.png
Saved YOLO annotations to D:/y